In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time, os

# ─── CONFIG ─────────────────────────────────────────────────────────────
URL               = "https://eproc.rajasthan.gov.in/nicgep/app?page=WebTenderStatusLists&service=page"
DEPARTMENT_VALUE  = "154"   # ← set this to the Assam-specific value
DEPARTMENT_NAME   = "PHED-CE(ADMIN) AND TMRWSSMB,JAIPUR"      # will be populated at runtime
FROM_DATE         = "01/07/2024"
TO_DATE           = "31/07/2025"
OUTPUT_XLS        = "Rajasthan_PAST_PHED-CE(ADMIN) AND TMRWSSMB,JAIPUR.xlsx"

# ─── BROWSER SETUP ──────────────────────────────────────────────────────
driver = webdriver.Chrome()
driver.maximize_window()
driver.get(URL)
time.sleep(2)

# ─── PICK DEPARTMENT ────────────────────────────────────────────────────
select = Select(driver.find_element(By.ID, "OrganName"))
select.select_by_value(DEPARTMENT_VALUE)
DEPARTMENT_NAME = select.first_selected_option.text.strip()
print(f"🔎 Using Department: {DEPARTMENT_NAME}")

# ─── SET DATE FILTERS ───────────────────────────────────────────────────
for fld_id, val in [("publishedFromDate", FROM_DATE), ("publishedToDate", TO_DATE)]:
    el = driver.find_element(By.ID, fld_id)
    driver.execute_script("arguments[0].removeAttribute('readonly')", el)
    el.clear()
    el.send_keys(val)

# ─── CAPTCHA ───────────────────────────────────────────────────────────
png = "captcha_Rajasthan.png"
with open(png, "wb") as f:
    f.write(driver.find_element(By.ID, "captchaImage").screenshot_as_png)
print(f"✔ CAPTCHA saved to {png}. Please open it to read.")
captcha = input("Enter CAPTCHA: ")
driver.find_element(By.ID, "captchaText").send_keys(captcha)
driver.find_element(By.ID, "Search").click()
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "tabList")))

# ─── FIELD EXTRACTOR ────────────────────────────────────────────────────
def get_field_value(label):
    target = label.strip().lower()
    captions = driver.find_elements(By.XPATH, "//td[contains(@class,'caption')]")
    for cap in captions:
        if target in cap.text.strip().lower():
            try:
                return cap.find_element(By.XPATH, "following-sibling::td[1]").text.strip()
            except:
                return ""
    return ""

# ─── SCRAPE & PAGINATE ──────────────────────────────────────────────────
all_data = []

while True:
    rows = driver.find_elements(By.XPATH, "//table[@id='tabList']//tr[@class='even' or @class='odd']")
    for idx in range(len(rows)):
        try:
            # refresh row list each iteration
            rows = driver.find_elements(By.XPATH, "//table[@id='tabList']//tr[@class='even' or @class='odd']")
            row = rows[idx]
            cells = row.find_elements(By.TAG_NAME, "td")
            if len(cells) < 6:
                continue

            sno, tid, title, orgs, stage, status = [c.text.strip() for c in cells[:6]]

            # open detail popup
            row.find_element(By.XPATH, ".//td[last()]/a").click()
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "DirectLink"))).click()
            WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)
            driver.switch_to.window(driver.window_handles[-1])
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "table")))

            # extract fields
            record = {
                "Department":               DEPARTMENT_NAME,
                "S.No":                     sno.rstrip('.'),
                "Tender ID":                tid,
                "Title & Ref No.":          title,
                "Organisation Chain":       orgs,
                "Tender Stage":             stage,
                "Status":                   status,
                "Publish Date":             get_field_value("Publish Date"),
                "Bid Submission End":       get_field_value("Bid Submission End Date"),
                "Tender Value (₹)":         get_field_value("Tender Value"),
                "Location":                 get_field_value("Location"),
                "Pincode":                  get_field_value("Pincode"),
                "Period Of Work (Days)":    get_field_value("Period Of Work"),
                "Address":                  get_field_value("Address"),
            }
            all_data.append(record)
            print(f"✔ Row {sno}: {tid} extracted")

            # close detail and return to list
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//a[contains(text(),'Back')]"))
            ).click()
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "tabList")))

        except Exception as e:
            print(f"❌ Row {idx+1} error: {e}")
            driver.switch_to.window(driver.window_handles[0])
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "tabList")))

    # move to next page, if any
    try:
        driver.find_element(By.ID, "loadNext").click()
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "tabList")))
    except:
        break

# ─── EXPORT TO EXCEL ───────────────────────────────────────────────────
df = pd.DataFrame(all_data)
df.to_excel(OUTPUT_XLS, index=False)
print(f"\n✅ Done! Saved {len(all_data)} records to {OUTPUT_XLS}")

driver.quit()


🔎 Using Department: PHED-CE(ADMIN) AND TMRWSSMB,JAIPUR
✔ CAPTCHA saved to captcha_Rajasthan.png. Please open it to read.


Enter CAPTCHA:  XkYV2f



✅ Done! Saved 0 records to Rajasthan_PAST_PHED-CE(ADMIN) AND TMRWSSMB,JAIPUR.xlsx


In [2]:
import pandas as pd

# Save the collected tender data to Excel
df = pd.DataFrame(all_data)
df.to_excel("Tamilnadu_PAST_Rural Development and Panchayat Raj Department.xlsx", index=False)
print(f"✅ Saved {len(all_data)} records.")


✅ Saved 20758 records.


In [3]:
import IPython
from IPython.display import FileLink

FileLink("Tamilnadu_PAST_Rural Development and Panchayat Raj Department.xlsx")


C:\Users\Jaydeb\Tamilnadu_PAST_Rural Development and Panchayat Raj Department.xlsx

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException, NoSuchElementException, StaleElementReferenceException,
)
from datetime import datetime
import pandas as pd, os, re, shutil, time

# ───────────────── CONFIG ─────────────────
URL = ("https://etenders.kerala.gov.in/nicgep/app?page=WebTenderStatusLists&service=page")
DEPTS = [("Roads and Bridges Development Corporation Kerala","207")]

published_from = "01/07/2024"  # dd/MM/yyyy format
published_to = "31/07/2025"

# Clean department name to avoid spaces in filename
dept_name_clean = DEPTS[0][0].replace(" ", "_")
OUT_XLSX = f"Kerala_tenders_{dept_name_clean}_{datetime.today().strftime('%d_%m_%Y')}.xlsx"

DL_ROOT = os.path.join(os.getcwd(), "downloads")
os.makedirs(DL_ROOT, exist_ok=True)
_clean = lambda s: re.sub(r"[^\d.]", "", s.strip())

def dept_dir(name: str) -> str:
    p = os.path.join(DL_ROOT, name)
    os.makedirs(p, exist_ok=True)
    return p

def await_download(before, target_dir, timeout=40):
    WebDriverWait(drv, timeout).until(
        lambda _: any(f.endswith((".xls", ".xlsx", ".pdf")) and not f.endswith(".crdownload")
                      for f in set(os.listdir(DL_ROOT)) - before))
    f = next(x for x in os.listdir(DL_ROOT) if x not in before and not x.endswith(".crdownload"))
    shutil.move(os.path.join(DL_ROOT, f), os.path.join(target_dir, f))
    print(f"⬇️  Downloaded file: {f} → {target_dir}")

# ───────────── DRIVER ─────────────
opt = webdriver.ChromeOptions()
opt.add_experimental_option("prefs", {
    "download.default_directory": DL_ROOT,
    "download.prompt_for_download": False,
    "safebrowsing.enabled": True
})
drv = webdriver.Chrome(options=opt)
drv.maximize_window()
print(f"📅  {published_from} → {published_to}")

all_rows = []
try:
    drv.get(URL)
    time.sleep(2)

    # Manual CAPTCHA - save image and prompt user input
    open("captcha.png", "wb").write(
        drv.find_element(By.ID, "captchaImage").screenshot_as_png)
    try:
        os.startfile("captcha.png")  # Windows only; adjust for others
    except FileNotFoundError:
        pass
    drv.find_element(By.ID, "captchaText").send_keys(input("Enter CAPTCHA: "))
    print("✅ CAPTCHA solved.\n")

    main_tab = drv.current_window_handle

    # ───────────── MAIN LOOP ─────────────
    for dept, val in DEPTS:
        print(f"🔍 {dept}")
        Select(drv.find_element(By.ID, "OrganName")).select_by_value(val)

        for box, date in [("publishedFromDate", published_from),
                          ("publishedToDate", published_to)]:
            drv.execute_script(
                "document.getElementById(arguments[0]).removeAttribute('readonly');", box)
            elm = drv.find_element(By.ID, box)
            elm.clear()
            elm.send_keys(date)

        drv.find_element(By.ID, "Search").click()
        WebDriverWait(drv, 30).until(
            EC.presence_of_element_located((By.ID, "tabList")))

        page = 1
        while True:
            rows = drv.find_elements(By.XPATH, "//table[@id='tabList']//tr[td]")
            if len(rows) == 1 and "No Data" in rows[0].text:
                break

            for row in rows:
                try:
                    tds = row.find_elements(By.TAG_NAME, "td")
                    if len(tds) < 6:
                        continue
                    anchors = tds[5].find_elements(By.TAG_NAME, "a")
                    if not anchors:
                        continue  # skip heading rows

                    sno, tid = [tds[i].text.strip() for i in (0, 1)]

                    # open status in new tab
                    link = anchors[0]
                    drv.execute_script("arguments[0].setAttribute('target','_blank');", link)
                    link.send_keys(Keys.CONTROL + Keys.RETURN)
                    WebDriverWait(drv, 8).until(
                        lambda d: len(d.window_handles) > 1)
                    detail_tab = [h for h in drv.window_handles if h != main_tab][-1]
                    drv.switch_to.window(detail_tab)

                    # View More Details
                    WebDriverWait(drv, 8).until(
                        EC.element_to_be_clickable(
                            (By.XPATH, "//a[contains(text(),'View More Details')]"))
                    ).click()

                    # BOQ
                    remark = "No"
                    save_to = os.path.join(dept_dir(dept), tid)
                    os.makedirs(save_to, exist_ok=True)

                    try:
                        # direct link on detail
                        direct = drv.find_elements(
                            By.XPATH,
                            "//a[@id='BOQ' or contains(text(),'BOQ Comparative Chart') or "
                            "contains(@href,'FrontEndViewBidSummaryDetailsPrint')]")
                        if direct:
                            before = set(os.listdir(DL_ROOT))
                            direct[0].click()
                            await_download(before, save_to)
                            remark = "Yes"
                        else:
                            # stage summary pop-up
                            stage_link = WebDriverWait(drv, 6).until(
                                EC.element_to_be_clickable((
                                    By.XPATH, "//a[contains(@onclick,'openPopUp') or "
                                              "starts-with(@id,'DirectLink_')]")))
                            stage_link.send_keys(Keys.CONTROL + Keys.RETURN)
                            WebDriverWait(drv, 6).until(
                                lambda d: len(d.window_handles) > 2)
                            pop = [h for h in drv.window_handles
                                   if h not in (main_tab, detail_tab)][0]
                            drv.switch_to.window(pop)
                            chart = WebDriverWait(drv, 8).until(
                                EC.element_to_be_clickable(
                                    (By.XPATH, "//*[@id='BOQ' or starts-with(@id,'CHART')]")))
                            before = set(os.listdir(DL_ROOT))
                            chart.click()
                            await_download(before, save_to)
                            remark = "Yes"
                    except Exception:
                        pass

                    # close extra tabs
                    for h in [h for h in drv.window_handles if h != main_tab]:
                        drv.switch_to.window(h)
                        drv.close()
                    drv.switch_to.window(main_tab)

                    all_rows.append({
                        "Department": dept,
                        "S.No": sno,
                        "Tender ID": tid,
                        "Remark": remark
                    })
                    print(f"   ➕ {tid} ({remark})")
                except (StaleElementReferenceException, NoSuchElementException) as e:
                    print(f"⚠️ Skipping a row due to: {e}")
                    continue
                except Exception as e:
                    print(f"⚠️ Error processing tender {tid}: {e}")
                    continue

            print(f"📃 Collected so far: {len(all_rows)} | page {page}")
            page += 1

            # next page
            try:
                drv.find_element(By.ID, "loadNext").click()
                WebDriverWait(drv, 8).until(
                    EC.presence_of_element_located((By.ID, "tabList")))
            except Exception:
                break

finally:
    if all_rows:
        pd.DataFrame(all_rows).to_excel(OUT_XLSX, index=False)
        print(f"\n✅ Rows written: {len(all_rows)} → {OUT_XLSX}")
        print(f"📂 BOQs → {DL_ROOT}/<Department>/<TenderID>/")
    else:
        print("\n⚠️ Script finished but collected ZERO rows — "
              "verify the date filter or that tenders exist.")
    drv.quit()


📅  01/07/2024 → 31/07/2025


Enter CAPTCHA:  8fz18t


✅ CAPTCHA solved.

🔍 Roads and Bridges Development Corporation Kerala
   ➕ 2025_KFL_779750_1 (No)
   ➕ 2025_KFL_779599_1 (No)
   ➕ 2025_KFL_779582_1 (No)
   ➕ 2025_KFL_779073_1 (No)
   ➕ 2025_KFL_779060_1 (No)
   ➕ 2025_KFL_778632_1 (No)
   ➕ 2025_KFL_776491_1 (No)
   ➕ 2025_KFL_775928_1 (No)
   ➕ 2025_KFL_775640_1 (No)
   ➕ 2025_KFL_775207_1 (No)
📃 Collected so far: 10 | page 1
   ➕ 2025_KFL_774038_1 (No)
   ➕ 2025_KFL_772944_1 (No)
   ➕ 2025_KFL_772675_1 (No)
   ➕ 2025_KFL_772098_1 (No)
   ➕ 2025_KFL_769646_1 (No)
   ➕ 2025_KFL_767466_1 (No)
   ➕ 2025_KFL_765208_1 (No)
   ➕ 2025_KFL_763254_1 (No)
   ➕ 2025_KFL_762244_1 (No)
   ➕ 2025_KFL_761906_1 (No)
📃 Collected so far: 20 | page 2
   ➕ 2025_KFL_760470_1 (No)
⬇️  Downloaded file: boqcomparativechart - 2025-07-31T124223.583.xlsx → C:\Users\Jaydeb\downloads\Roads and Bridges Development Corporation Kerala\2025_KFL_758855_1
   ➕ 2025_KFL_758855_1 (Yes)
   ➕ 2025_KFL_757512_1 (No)
⬇️  Downloaded file: boqcomparativechart - 2025-07-31T12

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException, NoSuchElementException, StaleElementReferenceException,
)
from datetime import datetime
import pandas as pd, os, re, shutil, time

# ───────────────── CONFIG ─────────────────
URL = ("https://eproc.rajasthan.gov.in/nicgep/app?page=WebTenderStatusLists&service=page")
DEPTS = [("Jaipur Development Authority","118")]

published_from = "01/07/2024"  # dd/MM/yyyy format
published_to = "31/07/2025"

# Clean department name to avoid spaces in filename
dept_name_clean = DEPTS[0][0].replace(" ", "_")
OUT_XLSX = f"Rajasthan_tenders_{dept_name_clean}_{datetime.today().strftime('%d_%m_%Y')}.xlsx"

DL_ROOT = os.path.join(os.getcwd(), "downloads")
os.makedirs(DL_ROOT, exist_ok=True)
_clean = lambda s: re.sub(r"[^\d.]", "", s.strip())

def dept_dir(name: str) -> str:
    p = os.path.join(DL_ROOT, name)
    os.makedirs(p, exist_ok=True)
    return p

def await_download(before, target_dir, timeout=40):
    WebDriverWait(drv, timeout).until(
        lambda _: any(f.endswith((".xls", ".xlsx", ".pdf")) and not f.endswith(".crdownload")
                      for f in set(os.listdir(DL_ROOT)) - before))
    f = next(x for x in os.listdir(DL_ROOT) if x not in before and not x.endswith(".crdownload"))
    shutil.move(os.path.join(DL_ROOT, f), os.path.join(target_dir, f))
    print(f"⬇️  Downloaded file: {f} → {target_dir}")

# ───────────── DRIVER ─────────────
opt = webdriver.ChromeOptions()
opt.add_experimental_option("prefs", {
    "download.default_directory": DL_ROOT,
    "download.prompt_for_download": False,
    "safebrowsing.enabled": True
})
drv = webdriver.Chrome(options=opt)
drv.maximize_window()
print(f"📅  {published_from} → {published_to}")

all_rows = []
try:
    drv.get(URL)
    time.sleep(2)

    # Manual CAPTCHA - save image and prompt user input
    open("captcha.png", "wb").write(
        drv.find_element(By.ID, "captchaImage").screenshot_as_png)
    try:
        os.startfile("captcha.png")  # Windows only; adjust for others
    except FileNotFoundError:
        pass
    drv.find_element(By.ID, "captchaText").send_keys(input("Enter CAPTCHA: "))
    print("✅ CAPTCHA solved.\n")

    main_tab = drv.current_window_handle

    # ───────────── MAIN LOOP ─────────────
    for dept, val in DEPTS:
        print(f"🔍 {dept}")
        Select(drv.find_element(By.ID, "OrganName")).select_by_value(val)

        for box, date in [("publishedFromDate", published_from),
                          ("publishedToDate", published_to)]:
            drv.execute_script(
                "document.getElementById(arguments[0]).removeAttribute('readonly');", box)
            elm = drv.find_element(By.ID, box)
            elm.clear()
            elm.send_keys(date)

        drv.find_element(By.ID, "Search").click()
        WebDriverWait(drv, 30).until(
            EC.presence_of_element_located((By.ID, "tabList")))

        page = 1
        while True:
            rows = drv.find_elements(By.XPATH, "//table[@id='tabList']//tr[td]")
            if len(rows) == 1 and "No Data" in rows[0].text:
                break

            for row in rows:
                try:
                    tds = row.find_elements(By.TAG_NAME, "td")
                    if len(tds) < 6:
                        continue
                    anchors = tds[5].find_elements(By.TAG_NAME, "a")
                    if not anchors:
                        continue  # skip heading rows

                    sno, tid = [tds[i].text.strip() for i in (0, 1)]

                    # open status in new tab
                    link = anchors[0]
                    drv.execute_script("arguments[0].setAttribute('target','_blank');", link)
                    link.send_keys(Keys.CONTROL + Keys.RETURN)
                    WebDriverWait(drv, 8).until(
                        lambda d: len(d.window_handles) > 1)
                    detail_tab = [h for h in drv.window_handles if h != main_tab][-1]
                    drv.switch_to.window(detail_tab)

                    # View More Details
                    WebDriverWait(drv, 8).until(
                        EC.element_to_be_clickable(
                            (By.XPATH, "//a[contains(text(),'View More Details')]"))
                    ).click()

                    # BOQ
                    remark = "No"
                    save_to = os.path.join(dept_dir(dept), tid)
                    os.makedirs(save_to, exist_ok=True)

                    try:
                        # direct link on detail
                        direct = drv.find_elements(
                            By.XPATH,
                            "//a[@id='BOQ' or contains(text(),'BOQ Comparative Chart') or "
                            "contains(@href,'FrontEndViewBidSummaryDetailsPrint')]")
                        if direct:
                            before = set(os.listdir(DL_ROOT))
                            direct[0].click()
                            await_download(before, save_to)
                            remark = "Yes"
                        else:
                            # stage summary pop-up
                            stage_link = WebDriverWait(drv, 6).until(
                                EC.element_to_be_clickable((
                                    By.XPATH, "//a[contains(@onclick,'openPopUp') or "
                                              "starts-with(@id,'DirectLink_')]")))
                            stage_link.send_keys(Keys.CONTROL + Keys.RETURN)
                            WebDriverWait(drv, 6).until(
                                lambda d: len(d.window_handles) > 2)
                            pop = [h for h in drv.window_handles
                                   if h not in (main_tab, detail_tab)][0]
                            drv.switch_to.window(pop)
                            chart = WebDriverWait(drv, 8).until(
                                EC.element_to_be_clickable(
                                    (By.XPATH, "//*[@id='BOQ' or starts-with(@id,'CHART')]")))
                            before = set(os.listdir(DL_ROOT))
                            chart.click()
                            await_download(before, save_to)
                            remark = "Yes"
                    except Exception:
                        pass

                    # close extra tabs
                    for h in [h for h in drv.window_handles if h != main_tab]:
                        drv.switch_to.window(h)
                        drv.close()
                    drv.switch_to.window(main_tab)

                    all_rows.append({
                        "Department": dept,
                        "S.No": sno,
                        "Tender ID": tid,
                        "Remark": remark
                    })
                    print(f"   ➕ {tid} ({remark})")
                except (StaleElementReferenceException, NoSuchElementException) as e:
                    print(f"⚠️ Skipping a row due to: {e}")
                    continue
                except Exception as e:
                    print(f"⚠️ Error processing tender {tid}: {e}")
                    continue

            print(f"📃 Collected so far: {len(all_rows)} | page {page}")
            page += 1

            # next page
            try:
                drv.find_element(By.ID, "loadNext").click()
                WebDriverWait(drv, 8).until(
                    EC.presence_of_element_located((By.ID, "tabList")))
            except Exception:
                break

finally:
    if all_rows:
        pd.DataFrame(all_rows).to_excel(OUT_XLSX, index=False)
        print(f"\n✅ Rows written: {len(all_rows)} → {OUT_XLSX}")
        print(f"📂 BOQs → {DL_ROOT}/<Department>/<TenderID>/")
    else:
        print("\n⚠️ Script finished but collected ZERO rows — "
              "verify the date filter or that tenders exist.")
    drv.quit()


📅  01/07/2024 → 31/07/2025


Enter CAPTCHA:  PA6bjz


✅ CAPTCHA solved.

🔍 Jaipur Development Authority
   ➕ 2025_JDAJP_491491_1 (No)
   ➕ 2025_JDAJP_491463_1 (No)
   ➕ 2025_JDAJP_491273_1 (No)
   ➕ 2025_JDAJP_490974_1 (No)
   ➕ 2025_JDAJP_490910_1 (No)
   ➕ 2025_JDAJP_490906_1 (No)
   ➕ 2025_JDAJP_490903_1 (No)
   ➕ 2025_JDAJP_490394_1 (No)
   ➕ 2025_JDAJP_490336_1 (No)
   ➕ 2025_JDAJP_490292_1 (No)
📃 Collected so far: 10 | page 1
   ➕ 2025_JDAJP_490251_1 (No)
   ➕ 2025_JDAJP_489811_1 (No)
   ➕ 2025_JDAJP_489675_1 (No)
   ➕ 2025_JDAJP_489368_1 (No)
   ➕ 2025_JDAJP_489074_1 (No)
   ➕ 2025_JDAJP_488830_1 (No)
⬇️  Downloaded file: boqcomparativechart - 2025-08-02T143132.040.xlsx → C:\Users\Jaydeb\downloads\Jaipur Development Authority\2025_JDAJP_488811_1
   ➕ 2025_JDAJP_488811_1 (Yes)
   ➕ 2025_JDAJP_488710_1 (No)
⬇️  Downloaded file: boqcomparativechart - 2025-08-02T143143.446.xlsx → C:\Users\Jaydeb\downloads\Jaipur Development Authority\2025_JDAJP_488656_1
   ➕ 2025_JDAJP_488656_1 (Yes)
   ➕ 2025_JDAJP_488612_1 (No)
📃 Collected so far: 20

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time, os

# ─── CONFIG ─────────────────────────────────────────────────────────────
URL               = "https://wbtenders.gov.in/nicgep/app?page=WebTenderStatusLists&service=page"
DEPARTMENT_VALUE  = "216"   # ← set this to the Assam-specific value
DEPARTMENT_NAME   = "PWD"      # will be populated at runtime
FROM_DATE         = "01/07/2024"
TO_DATE           = "31/07/2025"
OUTPUT_XLS        = "WB_PAST_PWD.xlsx"

# ─── BROWSER SETUP ──────────────────────────────────────────────────────
driver = webdriver.Chrome()
driver.maximize_window()
driver.get(URL)
time.sleep(2)

# ─── PICK DEPARTMENT ────────────────────────────────────────────────────
select = Select(driver.find_element(By.ID, "OrganName"))
select.select_by_value(DEPARTMENT_VALUE)
DEPARTMENT_NAME = select.first_selected_option.text.strip()
print(f"🔎 Using Department: {DEPARTMENT_NAME}")

# ─── SET DATE FILTERS ───────────────────────────────────────────────────
for fld_id, val in [("publishedFromDate", FROM_DATE), ("publishedToDate", TO_DATE)]:
    el = driver.find_element(By.ID, fld_id)
    driver.execute_script("arguments[0].removeAttribute('readonly')", el)
    el.clear()
    el.send_keys(val)

# ─── CAPTCHA ───────────────────────────────────────────────────────────
png = "captcha_WB.png"
with open(png, "wb") as f:
    f.write(driver.find_element(By.ID, "captchaImage").screenshot_as_png)
print(f"✔ CAPTCHA saved to {png}. Please open it to read.")
captcha = input("Enter CAPTCHA: ")
driver.find_element(By.ID, "captchaText").send_keys(captcha)
driver.find_element(By.ID, "Search").click()
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "tabList")))

# ─── FIELD EXTRACTOR ────────────────────────────────────────────────────
def get_field_value(label):
    target = label.strip().lower()
    captions = driver.find_elements(By.XPATH, "//td[contains(@class,'caption')]")
    for cap in captions:
        if target in cap.text.strip().lower():
            try:
                return cap.find_element(By.XPATH, "following-sibling::td[1]").text.strip()
            except:
                return ""
    return ""

# ─── SCRAPE & PAGINATE ──────────────────────────────────────────────────
all_data = []

while True:
    rows = driver.find_elements(By.XPATH, "//table[@id='tabList']//tr[@class='even' or @class='odd']")
    for idx in range(len(rows)):
        try:
            # refresh row list each iteration
            rows = driver.find_elements(By.XPATH, "//table[@id='tabList']//tr[@class='even' or @class='odd']")
            row = rows[idx]
            cells = row.find_elements(By.TAG_NAME, "td")
            if len(cells) < 6:
                continue

            sno, tid, title, orgs, stage, status = [c.text.strip() for c in cells[:6]]

            # open detail popup
            row.find_element(By.XPATH, ".//td[last()]/a").click()
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "DirectLink"))).click()
            WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)
            driver.switch_to.window(driver.window_handles[-1])
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "table")))

            # extract fields
            record = {
                "Department":               DEPARTMENT_NAME,
                "S.No":                     sno.rstrip('.'),
                "Tender ID":                tid,
                "Title & Ref No.":          title,
                "Organisation Chain":       orgs,
                "Tender Stage":             stage,
                "Status":                   status,
                "Publish Date":             get_field_value("Publish Date"),
                "Bid Submission End":       get_field_value("Bid Submission End Date"),
                "Tender Value (₹)":         get_field_value("Tender Value"),
                "Location":                 get_field_value("Location"),
                "Pincode":                  get_field_value("Pincode"),
                "Period Of Work (Days)":    get_field_value("Period Of Work"),
                "Address":                  get_field_value("Address"),
            }
            all_data.append(record)
            print(f"✔ Row {sno}: {tid} extracted")

            # close detail and return to list
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//a[contains(text(),'Back')]"))
            ).click()
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "tabList")))

        except Exception as e:
            print(f"❌ Row {idx+1} error: {e}")
            driver.switch_to.window(driver.window_handles[0])
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "tabList")))

    # move to next page, if any
    try:
        driver.find_element(By.ID, "loadNext").click()
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "tabList")))
    except:
        break

# ─── EXPORT TO EXCEL ───────────────────────────────────────────────────
df = pd.DataFrame(all_data)
df.to_excel(OUTPUT_XLS, index=False)
print(f"\n✅ Done! Saved {len(all_data)} records to {OUTPUT_XLS}")

driver.quit()


🔎 Using Department: PWD
✔ CAPTCHA saved to captcha_WB.png. Please open it to read.


Enter CAPTCHA:  Yd22DJ


✔ Row 1.: 2025_PWD_885689_1 extracted
✔ Row 2.: 2025_PWD_885514_1 extracted
✔ Row 3.: 2025_PWD_885448_9 extracted
✔ Row 4.: 2025_PWD_885448_8 extracted
✔ Row 5.: 2025_PWD_885448_7 extracted
✔ Row 6.: 2025_PWD_885448_6 extracted
✔ Row 7.: 2025_PWD_885448_5 extracted
✔ Row 8.: 2025_PWD_885448_4 extracted
✔ Row 9.: 2025_PWD_885448_3 extracted
✔ Row 10.: 2025_PWD_885448_2 extracted
✔ Row 11.: 2025_PWD_885448_10 extracted
✔ Row 12.: 2025_PWD_885448_1 extracted
✔ Row 13.: 2025_PWD_885444_1 extracted
✔ Row 14.: 2025_PWD_885435_1 extracted
✔ Row 15.: 2025_PWD_885262_1 extracted
✔ Row 16.: 2025_PWD_885205_1 extracted
✔ Row 17.: 2025_PWD_885171_1 extracted
✔ Row 18.: 2025_PWD_885130_2 extracted
✔ Row 19.: 2025_PWD_885130_1 extracted
✔ Row 20.: 2025_PWD_885051_4 extracted
✔ Row 21.: 2025_PWD_885051_3 extracted
✔ Row 22.: 2025_PWD_885051_2 extracted
✔ Row 23.: 2025_PWD_885051_1 extracted
✔ Row 24.: 2025_PWD_885015_1 extracted
✔ Row 25.: 2025_PWD_884985_2 extracted
✔ Row 26.: 2025_PWD_884985_1 extr

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException, NoSuchElementException, StaleElementReferenceException,
)
from datetime import datetime
import pandas as pd, os, re, shutil, time

# ───────────────── CONFIG ─────────────────
URL = ("https://wbtenders.gov.in/nicgep/app?page=WebTenderStatusLists&service=page")
DEPTS = [("KOLKATA MUNICIPAL CORPORATION","168")]

published_from = "01/07/2024"  # dd/MM/yyyy format
published_to = "31/07/2025"

# Clean department name to avoid spaces in filename
dept_name_clean = DEPTS[0][0].replace(" ", "_")
OUT_XLSX = f"WB_tenders_{dept_name_clean}_{datetime.today().strftime('%d_%m_%Y')}.xlsx"

DL_ROOT = os.path.join(os.getcwd(), "downloads")
os.makedirs(DL_ROOT, exist_ok=True)
_clean = lambda s: re.sub(r"[^\d.]", "", s.strip())

def dept_dir(name: str) -> str:
    p = os.path.join(DL_ROOT, name)
    os.makedirs(p, exist_ok=True)
    return p

def await_download(before, target_dir, timeout=40):
    WebDriverWait(drv, timeout).until(
        lambda _: any(f.endswith((".xls", ".xlsx", ".pdf")) and not f.endswith(".crdownload")
                      for f in set(os.listdir(DL_ROOT)) - before))
    f = next(x for x in os.listdir(DL_ROOT) if x not in before and not x.endswith(".crdownload"))
    shutil.move(os.path.join(DL_ROOT, f), os.path.join(target_dir, f))
    print(f"⬇️  Downloaded file: {f} → {target_dir}")

# ───────────── DRIVER ─────────────
opt = webdriver.ChromeOptions()
opt.add_experimental_option("prefs", {
    "download.default_directory": DL_ROOT,
    "download.prompt_for_download": False,
    "safebrowsing.enabled": True
})
drv = webdriver.Chrome(options=opt)
drv.maximize_window()
print(f"📅  {published_from} → {published_to}")

all_rows = []
try:
    drv.get(URL)
    time.sleep(2)

    # Manual CAPTCHA - save image and prompt user input
    open("captcha.png", "wb").write(
        drv.find_element(By.ID, "captchaImage").screenshot_as_png)
    try:
        os.startfile("captcha.png")  # Windows only; adjust for others
    except FileNotFoundError:
        pass
    drv.find_element(By.ID, "captchaText").send_keys(input("Enter CAPTCHA: "))
    print("✅ CAPTCHA solved.\n")

    main_tab = drv.current_window_handle

    # ───────────── MAIN LOOP ─────────────
    for dept, val in DEPTS:
        print(f"🔍 {dept}")
        Select(drv.find_element(By.ID, "OrganName")).select_by_value(val)

        for box, date in [("publishedFromDate", published_from),
                          ("publishedToDate", published_to)]:
            drv.execute_script(
                "document.getElementById(arguments[0]).removeAttribute('readonly');", box)
            elm = drv.find_element(By.ID, box)
            elm.clear()
            elm.send_keys(date)

        drv.find_element(By.ID, "Search").click()
        WebDriverWait(drv, 30).until(
            EC.presence_of_element_located((By.ID, "tabList")))

        page = 1
        while True:
            rows = drv.find_elements(By.XPATH, "//table[@id='tabList']//tr[td]")
            if len(rows) == 1 and "No Data" in rows[0].text:
                break

            for row in rows:
                try:
                    tds = row.find_elements(By.TAG_NAME, "td")
                    if len(tds) < 6:
                        continue
                    anchors = tds[5].find_elements(By.TAG_NAME, "a")
                    if not anchors:
                        continue  # skip heading rows

                    sno, tid = [tds[i].text.strip() for i in (0, 1)]

                    # open status in new tab
                    link = anchors[0]
                    drv.execute_script("arguments[0].setAttribute('target','_blank');", link)
                    link.send_keys(Keys.CONTROL + Keys.RETURN)
                    WebDriverWait(drv, 8).until(
                        lambda d: len(d.window_handles) > 1)
                    detail_tab = [h for h in drv.window_handles if h != main_tab][-1]
                    drv.switch_to.window(detail_tab)

                    # View More Details
                    WebDriverWait(drv, 8).until(
                        EC.element_to_be_clickable(
                            (By.XPATH, "//a[contains(text(),'View More Details')]"))
                    ).click()

                    # BOQ
                    remark = "No"
                    save_to = os.path.join(dept_dir(dept), tid)
                    os.makedirs(save_to, exist_ok=True)

                    try:
                        # direct link on detail
                        direct = drv.find_elements(
                            By.XPATH,
                            "//a[@id='BOQ' or contains(text(),'BOQ Comparative Chart') or "
                            "contains(@href,'FrontEndViewBidSummaryDetailsPrint')]")
                        if direct:
                            before = set(os.listdir(DL_ROOT))
                            direct[0].click()
                            await_download(before, save_to)
                            remark = "Yes"
                        else:
                            # stage summary pop-up
                            stage_link = WebDriverWait(drv, 6).until(
                                EC.element_to_be_clickable((
                                    By.XPATH, "//a[contains(@onclick,'openPopUp') or "
                                              "starts-with(@id,'DirectLink_')]")))
                            stage_link.send_keys(Keys.CONTROL + Keys.RETURN)
                            WebDriverWait(drv, 6).until(
                                lambda d: len(d.window_handles) > 2)
                            pop = [h for h in drv.window_handles
                                   if h not in (main_tab, detail_tab)][0]
                            drv.switch_to.window(pop)
                            chart = WebDriverWait(drv, 8).until(
                                EC.element_to_be_clickable(
                                    (By.XPATH, "//*[@id='BOQ' or starts-with(@id,'CHART')]")))
                            before = set(os.listdir(DL_ROOT))
                            chart.click()
                            await_download(before, save_to)
                            remark = "Yes"
                    except Exception:
                        pass

                    # close extra tabs
                    for h in [h for h in drv.window_handles if h != main_tab]:
                        drv.switch_to.window(h)
                        drv.close()
                    drv.switch_to.window(main_tab)

                    all_rows.append({
                        "Department": dept,
                        "S.No": sno,
                        "Tender ID": tid,
                        "Remark": remark
                    })
                    print(f"   ➕ {tid} ({remark})")
                except (StaleElementReferenceException, NoSuchElementException) as e:
                    print(f"⚠️ Skipping a row due to: {e}")
                    continue
                except Exception as e:
                    print(f"⚠️ Error processing tender {tid}: {e}")
                    continue

            print(f"📃 Collected so far: {len(all_rows)} | page {page}")
            page += 1

            # next page
            try:
                drv.find_element(By.ID, "loadNext").click()
                WebDriverWait(drv, 8).until(
                    EC.presence_of_element_located((By.ID, "tabList")))
            except Exception:
                break

finally:
    if all_rows:
        pd.DataFrame(all_rows).to_excel(OUT_XLSX, index=False)
        print(f"\n✅ Rows written: {len(all_rows)} → {OUT_XLSX}")
        print(f"📂 BOQs → {DL_ROOT}/<Department>/<TenderID>/")
    else:
        print("\n⚠️ Script finished but collected ZERO rows — "
              "verify the date filter or that tenders exist.")
    drv.quit()


📅  01/07/2024 → 31/07/2025


Enter CAPTCHA:  v58y57


✅ CAPTCHA solved.

🔍 KOLKATA MUNICIPAL CORPORATION
   ➕ 2025_KMC_885747_1 (No)
   ➕ 2025_KMC_885607_1 (No)
   ➕ 2025_KMC_885573_1 (No)
   ➕ 2025_KMC_885567_1 (No)
   ➕ 2025_KMC_885560_1 (No)
   ➕ 2025_KMC_885547_1 (No)
   ➕ 2025_KMC_885546_1 (No)
   ➕ 2025_KMC_885519_1 (No)
   ➕ 2025_KMC_885505_1 (No)
   ➕ 2025_KMC_885497_1 (No)
📃 Collected so far: 10 | page 1
   ➕ 2025_KMC_885476_2 (No)
   ➕ 2025_KMC_885446_1 (No)
   ➕ 2025_KMC_885434_1 (No)
   ➕ 2025_KMC_885425_1 (No)
   ➕ 2025_KMC_885365_1 (No)
   ➕ 2025_KMC_885362_1 (No)
   ➕ 2025_KMC_885330_1 (No)
   ➕ 2025_KMC_885327_1 (No)
   ➕ 2025_KMC_885304_1 (No)
   ➕ 2025_KMC_885247_1 (No)
📃 Collected so far: 20 | page 2
   ➕ 2025_KMC_885244_1 (No)
   ➕ 2025_KMC_885228_1 (No)
   ➕ 2025_KMC_885222_1 (No)
   ➕ 2025_KMC_885174_1 (No)
   ➕ 2025_KMC_885168_1 (No)
   ➕ 2025_KMC_885136_1 (No)
   ➕ 2025_KMC_885104_1 (No)
   ➕ 2025_KMC_885102_1 (No)
   ➕ 2025_KMC_885086_1 (No)
   ➕ 2025_KMC_885081_1 (No)
📃 Collected so far: 30 | page 3
   ➕ 2025_KMC

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException, NoSuchElementException, StaleElementReferenceException,
)
from datetime import datetime
import pandas as pd, os, re, shutil, time

# ───────────────── CONFIG ─────────────────
URL = ("https://wbtenders.gov.in/nicgep/app?page=WebTenderStatusLists&service=page")
DEPTS = [("WEST BENGAL INDUSTRIAL DEVELOPMENT CORPORATION","319")]

published_from = "01/07/2024"  # dd/MM/yyyy format
published_to = "31/07/2025"

# Clean department name to avoid spaces in filename
dept_name_clean = DEPTS[0][0].replace(" ", "_")
OUT_XLSX = f"WB_tenders_{dept_name_clean}_{datetime.today().strftime('%d_%m_%Y')}.xlsx"

DL_ROOT = os.path.join(os.getcwd(), "downloads")
os.makedirs(DL_ROOT, exist_ok=True)
_clean = lambda s: re.sub(r"[^\d.]", "", s.strip())

def dept_dir(name: str) -> str:
    p = os.path.join(DL_ROOT, name)
    os.makedirs(p, exist_ok=True)
    return p

def await_download(before, target_dir, timeout=40):
    WebDriverWait(drv, timeout).until(
        lambda _: any(f.endswith((".xls", ".xlsx", ".pdf")) and not f.endswith(".crdownload")
                      for f in set(os.listdir(DL_ROOT)) - before))
    f = next(x for x in os.listdir(DL_ROOT) if x not in before and not x.endswith(".crdownload"))
    shutil.move(os.path.join(DL_ROOT, f), os.path.join(target_dir, f))
    print(f"⬇️  Downloaded file: {f} → {target_dir}")

# ───────────── DRIVER ─────────────
opt = webdriver.ChromeOptions()
opt.add_experimental_option("prefs", {
    "download.default_directory": DL_ROOT,
    "download.prompt_for_download": False,
    "safebrowsing.enabled": True
})
drv = webdriver.Chrome(options=opt)
drv.maximize_window()
print(f"📅  {published_from} → {published_to}")

all_rows = []
try:
    drv.get(URL)
    time.sleep(2)

    # Manual CAPTCHA - save image and prompt user input
    open("captcha.png", "wb").write(
        drv.find_element(By.ID, "captchaImage").screenshot_as_png)
    try:
        os.startfile("captcha.png")  # Windows only; adjust for others
    except FileNotFoundError:
        pass
    drv.find_element(By.ID, "captchaText").send_keys(input("Enter CAPTCHA: "))
    print("✅ CAPTCHA solved.\n")

    main_tab = drv.current_window_handle

    # ───────────── MAIN LOOP ─────────────
    for dept, val in DEPTS:
        print(f"🔍 {dept}")
        Select(drv.find_element(By.ID, "OrganName")).select_by_value(val)

        for box, date in [("publishedFromDate", published_from),
                          ("publishedToDate", published_to)]:
            drv.execute_script(
                "document.getElementById(arguments[0]).removeAttribute('readonly');", box)
            elm = drv.find_element(By.ID, box)
            elm.clear()
            elm.send_keys(date)

        drv.find_element(By.ID, "Search").click()
        WebDriverWait(drv, 30).until(
            EC.presence_of_element_located((By.ID, "tabList")))

        page = 1
        while True:
            rows = drv.find_elements(By.XPATH, "//table[@id='tabList']//tr[td]")
            if len(rows) == 1 and "No Data" in rows[0].text:
                break

            for row in rows:
                try:
                    tds = row.find_elements(By.TAG_NAME, "td")
                    if len(tds) < 6:
                        continue
                    anchors = tds[5].find_elements(By.TAG_NAME, "a")
                    if not anchors:
                        continue  # skip heading rows

                    sno, tid = [tds[i].text.strip() for i in (0, 1)]

                    # open status in new tab
                    link = anchors[0]
                    drv.execute_script("arguments[0].setAttribute('target','_blank');", link)
                    link.send_keys(Keys.CONTROL + Keys.RETURN)
                    WebDriverWait(drv, 8).until(
                        lambda d: len(d.window_handles) > 1)
                    detail_tab = [h for h in drv.window_handles if h != main_tab][-1]
                    drv.switch_to.window(detail_tab)

                    # View More Details
                    WebDriverWait(drv, 8).until(
                        EC.element_to_be_clickable(
                            (By.XPATH, "//a[contains(text(),'View More Details')]"))
                    ).click()

                    # BOQ
                    remark = "No"
                    save_to = os.path.join(dept_dir(dept), tid)
                    os.makedirs(save_to, exist_ok=True)

                    try:
                        # direct link on detail
                        direct = drv.find_elements(
                            By.XPATH,
                            "//a[@id='BOQ' or contains(text(),'BOQ Comparative Chart') or "
                            "contains(@href,'FrontEndViewBidSummaryDetailsPrint')]")
                        if direct:
                            before = set(os.listdir(DL_ROOT))
                            direct[0].click()
                            await_download(before, save_to)
                            remark = "Yes"
                        else:
                            # stage summary pop-up
                            stage_link = WebDriverWait(drv, 6).until(
                                EC.element_to_be_clickable((
                                    By.XPATH, "//a[contains(@onclick,'openPopUp') or "
                                              "starts-with(@id,'DirectLink_')]")))
                            stage_link.send_keys(Keys.CONTROL + Keys.RETURN)
                            WebDriverWait(drv, 6).until(
                                lambda d: len(d.window_handles) > 2)
                            pop = [h for h in drv.window_handles
                                   if h not in (main_tab, detail_tab)][0]
                            drv.switch_to.window(pop)
                            chart = WebDriverWait(drv, 8).until(
                                EC.element_to_be_clickable(
                                    (By.XPATH, "//*[@id='BOQ' or starts-with(@id,'CHART')]")))
                            before = set(os.listdir(DL_ROOT))
                            chart.click()
                            await_download(before, save_to)
                            remark = "Yes"
                    except Exception:
                        pass

                    # close extra tabs
                    for h in [h for h in drv.window_handles if h != main_tab]:
                        drv.switch_to.window(h)
                        drv.close()
                    drv.switch_to.window(main_tab)

                    all_rows.append({
                        "Department": dept,
                        "S.No": sno,
                        "Tender ID": tid,
                        "Remark": remark
                    })
                    print(f"   ➕ {tid} ({remark})")
                except (StaleElementReferenceException, NoSuchElementException) as e:
                    print(f"⚠️ Skipping a row due to: {e}")
                    continue
                except Exception as e:
                    print(f"⚠️ Error processing tender {tid}: {e}")
                    continue

            print(f"📃 Collected so far: {len(all_rows)} | page {page}")
            page += 1

            # next page
            try:
                drv.find_element(By.ID, "loadNext").click()
                WebDriverWait(drv, 8).until(
                    EC.presence_of_element_located((By.ID, "tabList")))
            except Exception:
                break

finally:
    if all_rows:
        pd.DataFrame(all_rows).to_excel(OUT_XLSX, index=False)
        print(f"\n✅ Rows written: {len(all_rows)} → {OUT_XLSX}")
        print(f"📂 BOQs → {DL_ROOT}/<Department>/<TenderID>/")
    else:
        print("\n⚠️ Script finished but collected ZERO rows — "
              "verify the date filter or that tenders exist.")
    drv.quit()


📅  01/07/2024 → 31/07/2025


Enter CAPTCHA:  B5yxF7


✅ CAPTCHA solved.

🔍 WEST BENGAL INDUSTRIAL DEVELOPMENT CORPORATION
   ➕ 2025_WBIDC_884425_2 (No)
   ➕ 2025_WBIDC_883368_1 (No)
   ➕ 2025_WBIDC_881124_1 (No)
   ➕ 2025_WBIDC_879648_1 (No)
   ➕ 2025_WBIDC_879194_1 (No)
   ➕ 2025_WBIDC_876284_1 (No)
   ➕ 2025_WBIDC_875095_1 (No)
   ➕ 2025_WBIDC_874057_1 (No)
   ➕ 2025_WBIDC_871588_1 (No)
⬇️  Downloaded file: boqcomparativechart - 2025-08-12T095820.033.xlsx → C:\Users\Jaydeb\downloads\WEST BENGAL INDUSTRIAL DEVELOPMENT CORPORATION\2025_WBIDC_870397_1
   ➕ 2025_WBIDC_870397_1 (Yes)
📃 Collected so far: 10 | page 1
   ➕ 2025_WBIDC_871588_1 (No)
⬇️  Downloaded file: boqcomparativechart - 2025-08-12T095834.741.xlsx → C:\Users\Jaydeb\downloads\WEST BENGAL INDUSTRIAL DEVELOPMENT CORPORATION\2025_WBIDC_870397_1
   ➕ 2025_WBIDC_870397_1 (Yes)
⬇️  Downloaded file: boqcomparativechart - 2025-08-12T095836.957.xlsx → C:\Users\Jaydeb\downloads\WEST BENGAL INDUSTRIAL DEVELOPMENT CORPORATION\2025_WBIDC_870289_1
   ➕ 2025_WBIDC_870289_1 (Yes)
⬇️  Download